In [2]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import expon
from scipy.stats import uniform
from scipy.integrate import quad

In [3]:
def FOC(kj,ki,gammai,Vj,t,r,p):

    # The 'x' vector to be optimized is the vector of physician strategies, kj

    # Patient strategies
    Ui_aux = Vj*ki.reshape((-1,1)) - t     # Vj x ki
    ki_greater = np.array(ki.reshape((-1,1)) >= kj.reshape((1,-1))).astype(int)  # I x J boolean mask of ki > kj
    Ui = Ui_aux + ki_greater * gammai.reshape((-1,1))   # Vj x ki + gammai if ki > kj
    exp_Ui = np.exp(Ui)     # exp(Ui)
    αi = np.zeros((I,J))
    αi[Ui > 0] = exp_Ui[Ui > 0]
    sum_αi = np.sum(αi, axis = 1).reshape((-1,1))
    Si = np.divide(αi, sum_αi, where=sum_αi!=0, out=np.zeros_like(αi))   # I x J vector of patient strategies (logit)

    # Aggregates
    Qi = np.sum(Si, axis = 0)
    Xi = np.sum(ki_greater * Si, axis = 0)

    return r - p * Xi


In [4]:
# Number of doctors and patients
I = 100
J = 50

# Doctor parameters
Vj = np.random.random(J)
r =0.1
p = 100000*np.random.random(J)
t= 0.2

# Patient parameters
ki  = np.random.random(I)
gammai  = np.random.random(I)

# Doctor strategies guess
kj  = 0.5*np.ones(J)

In [5]:
def newton(f, Df, x_0, tol=1e-7, max_iter=100_000):
    x = x_0

    # Implement the zero-finding formula
    def q(x):
        Df_x = Df(x)  # Compute Df(x) once to avoid redundant calculations
        return np.where(Df_x == 0, 0, x - f(x) / Df_x)

    error = tol + 1
    n = 0
    while np.any(error > tol):
        n += 1
        if(n > max_iter):
            break
        y = q(x)
        error = np.where(y == 0, 0, np.abs(x - y))
        
        x = y
        x[x<0] = 0
        if all(x == 0):
            break
    return x, error


h = 0.001
f = lambda x: FOC(x, ki,gammai,Vj,t,r,p)
Df = lambda x: (f(x + h) - f(x))*(1/h)
k_guess = 0.5*np.ones(J)

a, b = newton(f, Df, k_guess, tol=1e-3, max_iter=1)

/var/folders/d9/f39ks_q941961hvlpvd2zymc0000gn/T/ipykernel_10543/2419968891.py:7: RuntimeWarning: divide by zero encountered in divide
  return np.where(Df_x == 0, 0, x - f(x) / Df_x)


In [6]:
Df(a)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:
a

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [23]:
def uij(ki,gammai,Vj,t,kj):
            "Utility patient i receives from doctor j"

            return Vj * ki - t + np.where(ki >= kj, gammai, 0)
        
def aij(u,λ):
            "Intermediate function to calculate sij"

            return np.where(u > 0, np.exp(λ*u), 0)
                
def sij(ki,gammai,Vj,t,kj,λ):    #It takes the J-sized vectors of all doctors' Vj and κj as arguments
            "Probability that patient i visits doctor j"

            u = uij(ki,gammai,Vj,t,kj)

            return aij(u,λ)/np.sum(aij(u,λ))

ki = 0.1*np.ones((1,2))
gammai = 0.5
λ = 1
t = 0
Vj = Vj.reshape((-1,1))
kj = kj.reshape((-1,1))

b = 5

F = expon(scale=1/b)
G = uniform
H = uniform(scale = 2)

f = lambda x: F.pdf(x)
g = lambda x: G.pdf(x)

df = -b * f


s_k = lambda x: sij(x,gammai,Vj,t,kj,λ)

l = kj.reshape(1,-1)

def ds_dk(ki,gammai,Vj,t,kj,λ):
    """Derivative d sij / d k"""

    u = uij(ki,gammai,Vj,t,kj)

    if np.sum(aij(u,λ)) == 0:
        return 0

    return np.divide(Vj * aij(u,λ) * np.sum(aij(u,λ)) - aij(u,λ) * np.sum(Vj * aij(u,λ)), (np.sum(aij(u,λ)))**2)

ds_dk(ki,gammai,Vj,t,kj,λ).shape


s_gamma = lambda x: np.diagonal(sij(k0.reshape(1,-1),x,V,t,k0,λ)) 
        # Evaluate sij at each doctor's κj, then take the diagonal, taking doctor j's sij for his own kj
        # This is a lambda function for a given value of γ

        gamma_integrand = lambda x: s_gamma(x) * g(x) # sij(γ) g(γ)

        n = 101
        x = np.linspace(G.ppf(0),G.ppf(1),n) # n-sized linspace across the domain of G

        results = []

        # Loop that evaluates sij(γ) g(γ) for 101 values of γ
        for i in x:
            result = gamma_integrand(i)
            results.append(result)

        results = np.array(results)

        mc_integral = np.sum(results, axis = 0)/n


d2Q_dk = aux_func(self, ds_sk, k0) + aux_func(self, self.sij, k0) * df




(50, 2)